## Autonomously send real sensor values to TTN

**Prerequisites:** You must have followed the [getting started with TTN notebook](../getting-started-ttn/getting-started-ttn.ipynb) before starting this one.

> We also consider that the application id is **iotlab-lorawan** and the device id is **iotlab-node** but you'll have to use the ids corresponding to your application and device configured in TTN.

In this notebook, you will write an autonomous LoRaWAN application with the following specifications:
- The application automatically connects to TTN at startup using the OTAA activation and a datarate index of 5
- In case of an activation issue, an error message is printed to the console and the application returns
- A message is printed to the console to confirm the success of the activation
- A message is sent periodically (check the duty cycle!) to the TTN backend. This message will first contain a basic string and in a second phase, it will
- A message is printed to the console each time the message is sent with success.

We provide a starting RIOT application with a [Makefile](Makefile) and a [main.c](./main.c). The [Makefile](Makefile) is already configured but you will have to edit the [main.c](./main.c) by following the instructions of this Notebook.

### Configure the TTN payload format

Before we start coding the application, we need to change how the data is displayed in the `Data` tab of your TTN application.

<figure style="text-align:center">
    <img src="images/application-tab-data.png" width="500" align="center"/><br/><br/>
    <figcaption><em>Application `Data` tab in the TTN console</em></figcaption>
</figure>

You can change this format of the payload used to display the content of an uplink message in the `Payload formats` tab:

<figure style="text-align:center">
    <img src="images/application-tab-payload-format.png" width="500" align="center"/><br/><br/>
    <figcaption><em>Application `Payload formats` tab in the TTN console</em></figcaption>
</figure>

1. Select `Custom` in the drop down button.
2. Copy the following code snippet in the `decoder` text edit:

```js
function Decoder(bytes, port) {
    var message = ""
    for (var i = 0; i < bytes.length; i++) {
        message += String.fromCharCode(bytes[i])
    }
    var decoded = {}
    decoded.message = message
    return decoded
}
```

### Implement an autonomous RIOT application

The LoRaWAN API of RIOT is documented online at http://doc.riot-os.org/group__pkg__semtech-loramac.html.

Let's edit the [main.c](./main.c).

1. Add the necessary includes under the comment `/* Add necessary include here */`:


```c9
#include "net/loramac.h"     /* core loramac definitions */
#include "semtech_loramac.h" /* package API */
```


2. Declare the global descriptor for the loramac stack:


```c
semtech_loramac_t loramac;  /* The loramac stack descriptor */
```


3. Configure the identifiers (application and device) and the application key. You can find them in the device overview on TTN.


```c
static const uint8_t deveui[LORAMAC_DEVEUI_LEN] = { 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00 };
static const uint8_t appeui[LORAMAC_APPEUI_LEN] = { 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00 };
static const uint8_t appkey[LORAMAC_APPKEY_LEN] = { 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00 };
```

**note:** in the device overview on TTN, it's possible to switch the representation of the EUIs and key from an hexadecimal representation (the default) to a C byte array representation (the one that interests us here): use the `<>` button to switch from one to the other and keep the MSB order.

4. At the beginning of the main function, initialize the loramac stack:

```c
    semtech_loramac_init(&loramac);
```

5. Then, configure the keys:

```c
    semtech_loramac_set_deveui(&loramac, deveui);
    semtech_loramac_set_appeui(&loramac, appeui);
    semtech_loramac_set_appkey(&loramac, appkey);
```

6. All devices are very close to the gateway in IoT-LAB, so we can configure a fast datarate. Let's use DR5:

```c
    semtech_loramac_set_dr(&loramac, 5);
```

7. Add the logic to join the network using the OTAA activation:

```c
    if (semtech_loramac_join(&loramac, LORAMAC_JOIN_OTAA) != SEMTECH_LORAMAC_JOIN_SUCCEEDED) {
        puts("Join procedure failed");
        return 1;
    }
    puts("Join procedure succeeded");
```

8. Finally, in the `while` loop (under the comment `/* send the message here */`), send the message or continue if the message couldn't be sent:

```c
        if (semtech_loramac_send(&loramac,
                                 (uint8_t *)message, strlen(message)) != SEMTECH_LORAMAC_TX_DONE) {
            printf("Cannot send message '%s'\n", message);
        }
        else {
            printf("Message '%s' sent\n", message);
        }
```

9. Let's verify that the application builds correctly:

In [ ]:
!make

## Start an experiment on IoT-LAB

Now that we have a ready application, we can try it on the real hardware provided remotely by IoT-LAB.

1. Submit an experiment with one LoRa device on IoT-LAB:

In [ ]:
!iotlab-experiment submit -n "ttn-sensors" -d 120 -l 1,archi=st-lrwan1:sx1276+site=saclay

2. Wait for the experiment to be in the "Running" state:

In [ ]:
!iotlab-experiment wait --timeout 30 --cancel-on-timeout

**Note:** If the command above returns the message `Timeout reached, cancelling experiment <exp_id>`, try to re-submit your experiment later.

3. Open a new terminal with the menu `File > New > Terminal` and in the terminal, run the following command to connect to the serial link of the LoRa board:

**Keep this command running.**

4. Build and flash the application on the LoRa board:

In [ ]:
!make IOTLAB_NODE=auto-ssh flash

One the flashing is complete, in the terminal, you should see the messages corresponding to the join procedure ("Join procedure succeeed") and the messages sent every 20s.

If the device cannot join, verify the configured EUIs and the application key.

If everything works as expected, you should see the decoded "This is RIOT!" messages appear in the `Data` tab of your application:


<figure style="text-align:center">
    <img src="images/ttn-decoded-messages-received.png" width="600" align="center"/><br/><br/>
    <figcaption><em>Decoded messages received in the TTN console</em></figcaption>
</figure>

### Free up the resources

Since you finished the training, stop your experiment to free up the experiment nodes:

In [ ]:
!iotlab-experiment stop

The serial link connection through SSH will be closed automatically.